In [5]:
import re
import pandas as pd
from konlpy.tag import Komoran
import re
from dateutil import parser
from datetime import datetime, timedelta

In [2]:
data = pd.read_excel('adhd_data.xlsx')
data.head(5)

,sentence,syear,smonth,sday,shour,sminute,fyear,fmonth,fday,fhour,fminute,todo
0,3월 17일 9시에 헬스장 가기로 했어,2024.0,3.0,17.0,9.0,0.0,2024.0,3.0,17.0,9.0,0.0,헬스장
1,4월 5일 오후 3시에 친구와 점심 약속이 있어,2024.0,4.0,5.0,15.0,0.0,2024.0,4.0,5.0,15.0,0.0,친구와 점심 약속
2,5월 12일 오전 10시에 회사 세미나에 참석할 예정이야,2024.0,5.0,12.0,10.0,0.0,2024.0,5.0,12.0,10.0,0.0,회사 세미나
3,6월 8일 오후 2시에 친구의 생일 파티에 초대받았어,2024.0,6.0,8.0,14.0,0.0,2024.0,6.0,8.0,14.0,0.0,친구의 생일파티
4,7월 20일 오후 5시에 치과 진료 예약이 있어,2024.0,7.0,20.0,17.0,0.0,2024.0,7.0,20.0,17.0,0.0,치과 진료 예약


코드 초안

맨 마지막 코드 이용

In [ ]:
def extract_schedule_info(sentence):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)  # 특수문자 제거
    text = text.replace(" ", "")  # 모든 공백 제거
    
    # 시작 기간을 추출하는 정규표현식
    start_time_pattern = r'(?:(어제|오늘|내일|모레|\d+년))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후))?(?:(\d+)시)?(?:(\d+)분)?(?:부터)?' # 시작 기간 추출
    match = re.search(start_time_pattern, text)
    if match:
        syear = match.group(1) if match.group(1) else datetime.now().year  # 년도를 추출합니다. 년도가 없으면 기본값으로 2024를 사용합니다.
        smonth = match.group(2).strip("월") if match.group(2) else datetime.now().month
        sday = match.group(3).strip("일") if match.group(3) else datetime.now().day
        
        
        
        
        stime_period = match.group(4)
        shour = match.group(5) if match.group(5) else "0"
        if stime_period == '오후' and int(shour) < 12:
            shour = str(int(shour) + 12)
        if not stime_period:
            shour = str(int(shour))    
        sminute = match.group(6) if match.group(6) else "0"
        return syear, smonth, sday, stime_period, shour, sminute
        
    else:
        return None, None, None, None, None

In [28]:
def extract_schedule_info(sentence):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)  # 특수문자 제거
    text = text.replace(" ", "")  # 모든 공백 제거
    
    # 시작 기간을 추출하는 정규표현식
    start_time_pattern = r'(?:(어제|오늘|내일|모레|다음주|다다음주|\d+년))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후))?(?:(\d+)시)?(?:(반|\d+분))?(?:부터)?' # 시작 기간 추출
    match = re.search(start_time_pattern, text)
    if match:
        #syear = match.group(1) if match.group(1) else datetime.now().year  # 년도를 추출합니다. 년도가 없으면 기본값으로 2024를 사용합니다.
        smonth = match.group(2).strip("월") if match.group(2) else datetime.now().month
        sday = match.group(3).strip("일") if match.group(3) else datetime.now().day
        
        if match.group(1) == '오늘':
            syear = datetime.now().year
            smonth = datetime.now().month
            sday = datetime.now().day
        elif match.group(1) == '어제':
            syear = (datetime.now() - timedelta(days=1)).year
            smonth = (datetime.now() - timedelta(days=1)).month
            sday = (datetime.now() - timedelta(days=1)).day
        elif match.group(1) == '내일':
            syear = (datetime.now() + timedelta(days=1)).year
            smonth = (datetime.now() + timedelta(days=1)).month
            sday = (datetime.now() + timedelta(days=1)).day
        elif match.group(1) == '모레':
            syear = (datetime.now() + timedelta(days=2)).year
            smonth = (datetime.now() + timedelta(days=2)).month
            sday = (datetime.now() + timedelta(days=2)).day
        elif match.group(1) == '다음주':
            syear = (datetime.now() + timedelta(days=7)).year
            smonth = (datetime.now() + timedelta(days=7)).month
            sday = (datetime.now() + timedelta(days=7)).day
        elif match.group(1) == '다다음주':
            syear = (datetime.now() + timedelta(days=14)).year
            smonth = (datetime.now() + timedelta(days=14)).month
            sday = (datetime.now() + timedelta(days=14)).day
        elif match.group(1) and '년' in match.group(1):
            syear = match.group(1).strip("년")
        else:
            syear = datetime.now().year
            
        stime_period = match.group(4)
        shour = match.group(5) if match.group(5) else "0"
        if stime_period == '오후' and int(shour) < 12:
            shour = str(int(shour) + 12)
        if not stime_period:
            shour = str(int(shour))    
        sminute = match.group(6).strip("분") if match.group(6) else "0"
        if match.group(6) and '반' in match.group(6):
            sminute = 30
        
        # 종료 기간을 추출하는 정규표현식
        end_time_pattern = r'(?:(어제|오늘|내일|모레|다음주|다다음주|\d+년))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후))?(?:(\d+)시)?(?:(반|\d+분))?((까지|에))' # 종료 기간 추출
        end_match = re.search(end_time_pattern, text)
        if end_match:
            fyear = end_match.group(1) if end_match.group(1) else syear  # 종료 년도를 추출합니다.
            fmonth = end_match.group(2).strip("월") if end_match.group(2) else smonth
            fday = end_match.group(3).strip("일") if end_match.group(3) else sday
            ftime_period = end_match.group(4)
            fhour = end_match.group(5) if end_match.group(5) else "0"
            if ftime_period == '오후' and int(fhour) < 12:
                fhour = str(int(fhour) + 12)
            if not ftime_period:
                fhour = str(int(fhour))    
            fminute = end_match.group(6).strip("분") if end_match.group(6) else "0"
            if end_match.group(6) and '반' in end_match.group(6):
                fminute = 30
            
        else:
            fyear, fmonth, fday, ftime_period, fhour, fminute = syear, smonth, sday, stime_period, shour, sminute

        return syear, smonth, sday, stime_period, shour, sminute, fyear, fmonth, fday, ftime_period, fhour, fminute
    else:
        return None, None, None, None, None, None, None, None, None, None, None, None

In [29]:
a = extract_schedule_info("3월18일오후3시30분부터 오후4시40분까지 자라가 ㅎㅎ")
print(a)

(2024, '3', '18', '오후', '15', '30', 2024, '3', '18', '오후', '16', '40')
